In [120]:
# Import standard numeric and visualization libs
import numpy as np
import matplotlib.pyplot as plt

# Import my householder QR factorization implementation
# import sys
# sys.path.insert(0, "/home/masvgp/dev/byu_num_lin_alg_2022/src/")
# from factorizations.householder import house, formQ

In [121]:
# My implementation of householder factorization and it's associated utility functions
def e(i, length):
    """Generate a unit coordinate vector, i.e., Given an index i and a length, return a vector with a 1.0 in the ith position and 0.0 else.

    Args:
        i (int): Integer index in which 1.0 will be assigned  
        length (int): Length of the desired output vector

    Returns:
        arr: Unit coordinate vector with 1.0 in the ith position
    """
    e_i = np.zeros(length, dtype=np.float64)
    e_i[i] = 1.0
    return e_i

def reflector(v):
    """Given a unit vector v, return the householder reflector I - 2 * np.outer(v, v)

    Args:
        v (arr): Vector
    Returns:
        reflector (arr): A len(v) x len(v) array representing a householder reflector
    """
    I = np.eye(len(v))
    outer = np.outer(v, v)


    return (I - 2 * outer)

def house(A, reduced=False):
    """Compute the factor R of a QR factorization of an m x n matrix A with m >= .

    Args:
        A (arr): A numpy array of shape m x n
    Returns:
        R (arr): The upper diagonal matrix R in a QR factorization
    """
    m = A.shape[0] # Get row-dim of A
    n = A.shape[1] # Get col-dim of A
    W = np.zeros((m, n))
    #Q = np.eye(m, m)
    # V = np.zeros((m, n))

    # Cast A as type np.float64
    A = A.astype(np.float64)

    for k in range(n):
        # Get the first column of the (m-k+1, n-k+1) submatrix of A
        x = A[k:m, k]

        # Compute e_1, sign function, and the norm of x
        e_1 = e(0, len(x))
        sign = np.sign(x[0])
        norm_x = np.linalg.norm(x)

        # Compute vk, the vector reflected across the Householder hyperplane
        vk = (sign * norm_x * e_1) + x
        vk = vk / np.linalg.norm(vk)

        # Store vk in W
        W[k:m, k] = vk

        # Apply the reflector to the k:m, k:n submatrix of A to put
        # zeros below the diagonal of the kth column of A
        A[k:m, k:n] = reflector(vk) @ A[k:m, k:n]
    
    if reduced == True:
        # return np.around(A[:n, :n], decimals=8), W
        return A[:n, :n], W
    else:
        # Return full R
        return A, W
        # return np.around(A, decimals=8), W

def formQ(W, reduced=False):
    """Form Q from the lower diagonal vk's formed in the householder algorithm.

    Args:
        W (arr): An m x n matrix containing the vk in the kth column

    Returns:
        arr: Returns an m x m orthonormal matrix Q such that QR = A
    """
    m = W.shape[0]
    n = W.shape[1]
    Q = np.eye(m, m)
    for k in range(n):
            Qk = np.eye(m, m)
            Qk[k:m, k:m] = reflector(W[k:m, k])
            Q = Q @ Qk
    
    if reduced == True:
        # Return reduced Q
        return Q[:m, :n]
        # return np.around(Q[:m, :n], decimals=8)
    else:
        # Return full Q
        return Q
        # return np.around(Q, decimals=6)

# Problem 4
Use the result of Problem 3 to develop a Python function lq that takes as input a matrix $A$ and returns $Q$ and $L$ where $A = QL$ is a $QL$-decomposition, and test it with random matrices of sizes $10 \times 7$ and $75 \times 50$. Organize your code in a single Python notebook and include it in the PDF you turn in. By test your code, I mean that you compute the QL decomposition, that you verify $Q$ has orthonormal columns, you verify that $L$ is lower triangular, and that you compute $\begin{Vmatrix} A - QL \end{Vmatrix}_2$, where $A$ is a random matrix for wich you compute the $QL$ decomposition. I should be able to run your single file code and observe that you have done each of these verifications. I should also be able to see this by reading the PDF you turn in. To verify that $Q$ is orthogonal, you might just compute the maximum absolute value of the off diagonal entries of $Q^*Q$, for example.

### Solution:

In [122]:
def ql(A):
    """Compute the QL factorization of an m x n matrix A.       

    Args:
        A (arr): An m x n matrix
    Return:
        Q (arr): a unitary matrix
        L (arr): A lower triangular matrix
    """
    # Get dims of A
    m = A.shape[0]
    n = A.shape[1]

    # Define the reversal matrix K_n
    Kn = np.flip(np.eye(n), axis=0)

    # Compute reduced QR(AK_n) - \hat{Q} is m x n, \hat{R} is n x n.
    Rhat, W = house(A @ Kn, reduced=True)
    Qhat = formQ(W, reduced=True)
    
    # Compute Q = \hat{Q}K_n
    Q = Qhat @ Kn

    # Compute L = K_n\hat{R}K_n
    L = Kn @ Rhat @ Kn
    
    # return Q, L
    return Q, L

# Testing
1. Define matrices $A$ and $B$ of size $10 \times 7$ and $75 \times 50$, respectively. Then do the following:
2. Compute $QL(A)$ and $QL(B)$
3. Verify that $Q$ has orthonormal columns
4. Verify that $L$ is lower triangular
5. Compute $\begin{Vmatrix} A - QL \end{Vmatrix}_2$ and $\begin{Vmatrix} B - QL \end{Vmatrix}_2$, where $A$ and $B$ are the random matrices defined in (1).

### Testing for the $10 \times 7$ random matrix a

In [123]:
# 1. Define random matrix of size 10 x 7
A = np.random.normal(loc=0.0, scale=1.0, size=(10, 7))

In [124]:
# 2. Compute $QL(A)$
QA, LA = ql(A)

In [125]:
# 3. Verify that $Q$ has orthonormal columns
# Compute QA.T @ QA
QATQA = QA.T @ QA

# Check the diagonals are 1.0
print(f"The dimensions of $QA.T @ QA are: {QATQA.shape[0]} x {QATQA.shape[1]}")
print(f"Check that the diagonal entries of QA.T @ QA sum to {QATQA.shape[0]} by checking the sum of the diagonal entries. \n" + f"The sum is equal to {QATQA.shape[0]}: " + str(sum(np.around(np.diag(QATQA), decimals=10)) == QATQA.shape[0]))

# Set diagonal entries to 0.0
for i in range(QATQA.shape[0]):
    QATQA[i, i] = 0.0

# Compute the max absolute value of the the array QATQA with diagonal set to 0.0.
print("Check the max absolute value of the array after setting the diagonal entries to 0.0.\nThis effectively checks the max absolute value of the off-diagonal entries.\n" + "The max absolute value of the resulting array is: " + str(np.max(np.abs(QATQA))))

The dimensions of $QA.T @ QA are: 7 x 7
Check that the diagonal entries of QA.T @ QA sum to 7 by checking the sum of the diagonal entries. 
The sum is equal to 7: True
Check the max absolute value of the array after setting the diagonal entries to 0.0.
This effectively checks the max absolute value of the off-diagonal entries.
The max absolute value of the resulting array is: 3.2963142379103426e-16


In [126]:
# 4. Verify that $L$ is lower triangular
# Extract upper diagonal part of L
LA_upper_diag = [LA[i, j] for i in range(LA.shape[0]) for j in range(LA.shape[1]) if i < j]
# Extract the lower diagonal part of L
LA_lower_diag = [LA[i, j] for i in range(LA.shape[0]) for j in range(LA.shape[1]) if i >= j]

print("The max absolute value of the upper diagonal entries of LA is: " + str(np.max(np.abs(LA_upper_diag))))
print("The max absolute value of the lower diagonal entries of LA is: \n" + str(np.max(np.abs(LA_lower_diag))))

The max absolute value of the upper diagonal entries of LA is: 4.2312988466972233e-16
The max absolute value of the lower diagonal entries of LA is: 
3.0734874283890266


In [127]:
# 5. Compute $\begin{Vmatrix} A - QL \end{Vmatrix}_2$, where $A$ is the random matrices defined in (1).
print("The L2-norm of A - QL is: " + str(np.linalg.norm(A - QA @ LA)))

The L2-norm of A - QL is: 3.831465111543962e-15


### Testing for the $75 \times 50$ random matrix $B$.

In [128]:
# 1. Define random matrix of size 75 x 50
B = np.random.normal(loc=0.0, scale=1.0, size=(75, 50))

In [129]:
# Compute QL for the 75 x 50 matrix B
QB, LB = ql(B)

In [130]:
# 3. Verify that $Q$ has orthonormal columns
# Compute QA.T @ QA
QBTQB = QB.T @ QB

# Check the diagonals are 1.0
print(f"The dimensions of $QB.T @ QB are: {QBTQB.shape[0]} x {QBTQB.shape[1]}")
print(f"Check that the diagonal entries of QB.T @ QB sum to {QBTQB.shape[0]} by checking the sum of the diagonal entries. \n" + f"The sum is equal to {QBTQB.shape[0]}: " + str(sum(np.around(np.diag(QBTQB), decimals=10)) == QBTQB.shape[0]))

# Set diagonal entries to 0.0
for i in range(QBTQB.shape[0]):
    QBTQB[i, i] = 0.0

# Compute the max absolute value of the the array QATQA with diagonal set to 0.0.
print("Check the max absolute value of the array after setting the diagonal entries to 0.0.\nThis effectively checks the max absolute value of the off-diagonal entries.\n" + "The max absolute value of the resulting array is: " + str(np.max(np.abs(QBTQB))))

The dimensions of $QB.T @ QB are: 50 x 50
Check that the diagonal entries of QB.T @ QB sum to 50 by checking the sum of the diagonal entries. 
The sum is equal to 50: True
Check the max absolute value of the array after setting the diagonal entries to 0.0.
This effectively checks the max absolute value of the off-diagonal entries.
The max absolute value of the resulting array is: 8.721678135614107e-16


In [1]:
# 4. Verify that $L$ is lower triangular
# Extract upper diagonal part of L
LB_upper_diag = [LB[i, j] for i in range(LB.shape[0]) for j in range(LB.shape[1]) if i < j]
# Extract the lower diagonal part of L
LB_lower_diag = [LB[i, j] for i in range(LB.shape[0]) for j in range(LB.shape[1]) if i >= j]

print("The max absolute value of the upper diagonal entries of LB is: " + str(np.max(np.abs(LB_upper_diag))))
print("The max absolute value of the lower diagonal entries of LB is: \n" + str(np.max(np.abs(LB_lower_diag))))

NameError: name 'LB' is not defined

In [132]:
# 5. Compute $\begin{Vmatrix} B - QL \end{Vmatrix}_2$ where $B$ is the random matrices defined in (1).
print("The L2-norm of B - QL is: " + str(np.linalg.norm(B - QB @ LB)))

The L2-norm of B - QL is: 1.0843623836693697e-13
